In [1]:
import os
import sys
import subprocess
import random
import shutil

# ==========================================
# 1. AUTO-INSTALL REQUIREMENTS ON PYTHON 3.11
# ==========================================
# We define the path to your Python 3.11 executable based on your screenshot
PYTHON_311_PATH = r"C:\Users\Dell\AppData\Local\Programs\Python\Python311\python.exe"

# If we are NOT currently running on Python 3.11, we warn the user
if sys.executable != PYTHON_311_PATH:
    print(f"⚠️ WARNING: Current Kernel is {sys.version.split()[0]}")
    print(f"   Spark needs Python 3.11. Configuring Spark to use: {PYTHON_311_PATH}")

    # Ensure pyspark is installed on the 3.11 environment specifically
    print("   Verifying libraries on Python 3.11...")
    try:
        subprocess.check_call([PYTHON_311_PATH, "-m", "pip", "install", "pyspark", "mongo-spark-connector", "pymongo", "pandas"], 
                              stdout=subprocess.DEVNULL)
        print("   ✅ Libraries verified on Python 3.11.")
    except Exception as e:
        print(f"   ⚠️ Could not auto-install libraries: {e}")

# ==========================================
# 2. ENVIRONMENT CONFIGURATION
# ==========================================
# Paths verified from your previous successes
os.environ['JAVA_HOME'] = r"C:\jdk-11"
os.environ['HADOOP_HOME'] = r"C:\hadoop"

# CRITICAL FIX: Force Spark to use Python 3.11 worker even if the notebook is 3.13
os.environ['PYSPARK_PYTHON'] = PYTHON_311_PATH
os.environ['PYSPARK_DRIVER_PYTHON'] = PYTHON_311_PATH

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, size, split, current_timestamp, desc, avg, count
from pyspark.sql.types import StringType, DoubleType, ArrayType, IntegerType

# Local folder for temp data (Anti-Freeze fix)
warehouse_location = os.path.abspath("spark-warehouse")
ivy_location = os.path.abspath("spark-ivy") 

print("\n>>> Initializing Spark Session...")
print("    (If this takes >2 mins, it is downloading MongoDB connectors)")

spark = SparkSession.builder \
    .appName("BlockchainDWBuilder") \
    .master("local[1]") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.hadoop.home.dir", r"C:\hadoop") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.2.0") \
    .config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/aci.articles") \
    .config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/aci.fact_publications") \
    .config("spark.jars.ivy", ivy_location) \
    .getOrCreate()

print(f"✅ Spark Session Active: v{spark.version}")
spark.sparkContext.setLogLevel("WARN")

# ==========================================
# 3. DEFINE UDFS (Simulate BI Data)
# ==========================================
def simulate_quartile():
    return random.choices(["Q1", "Q2", "Q3", "Q4"], weights=[25, 35, 25, 15])[0]

def simulate_country():
    countries = ["USA", "China", "India", "UK", "France", "Germany", "Morocco", "Canada", "Japan"]
    weights = [20, 18, 15, 10, 8, 8, 5, 5, 11]
    return random.choices(countries, weights=weights)[0]

def simulate_impact_score():
    return round(random.uniform(0.5, 15.0), 2)

def extract_keywords(title):
    stopwords = ["the", "for", "and", "with", "based", "using", "approach", "blockchain", "analysis"]
    if not title: return []
    words = title.lower().replace("-", " ").split()
    return [w.capitalize() for w in words if len(w) > 4 and w not in stopwords]

udf_quartile = udf(simulate_quartile, StringType())
udf_country = udf(simulate_country, StringType())
udf_impact = udf(simulate_impact_score, DoubleType())
udf_keywords = udf(extract_keywords, ArrayType(StringType()))

# ==========================================
# 4. ETL PROCESS
# ==========================================
print("\n>>> 1. READING FROM MONGODB (aci.articles)...")
try:
    raw_df = spark.read.format("mongodb").load()
    count_raw = raw_df.count()
    print(f"   - Loaded {count_raw} raw articles.")
    
    if count_raw == 0:
        print("⚠️ WARNING: MongoDB collection is empty! Run scraping first.")
except Exception as e:
    print(f"❌ Error reading MongoDB: {e}")
    spark.stop()
    sys.exit(1)

print("\n>>> 2. TRANSFORMING DATA...")
# Clean Authors: "Name1;\nName2" -> Array ["Name1", "Name2"]
df_clean = raw_df.withColumn("authors_clean", split(col("authors"), ";\\\\n|;"))

# Build Fact Table
df_dw = df_clean.withColumn("nb_authors", size(col("authors_clean"))) \
    .withColumn("quartile", udf_quartile()) \
    .withColumn("country", udf_country()) \
    .withColumn("impact_score", udf_impact()) \
    .withColumn("citations", (col("impact_score") * 10).cast(IntegerType())) \
    .withColumn("generated_keywords", udf_keywords(col("title"))) \
    .withColumn("etl_timestamp", current_timestamp())

# Drop raw columns
df_final = df_dw.drop("authors", "_id")

print("\n>>> 3. LOADING TO DATA WAREHOUSE (aci.fact_publications)...")
df_final.write.format("mongodb").mode("overwrite").save()
print("   - Success. Data stored in MongoDB.")

# ==========================================
# 5. HDFS ARBORESCENCE (The Deliverable)
# ==========================================
hdfs_output_path = "hdfs_data"
print(f"\n>>> 4. GENERATING HDFS STRUCTURE AT: {os.path.abspath(hdfs_output_path)}")

# Write to Parquet, partitioned by Country
df_final.write.mode("overwrite").partitionBy("country").parquet(hdfs_output_path)
print("   - HDFS Folder Structure Created ✅")

# ==========================================
# 6. ANALYTICS PREVIEW
# ==========================================
print("\n" + "="*40)
print("        SPARK ANALYSIS RESULTS        ")
print("="*40)

print("\n📊 1. Top Countries by Impact Factor:")
df_final.groupBy("country") \
    .agg(avg("impact_score").alias("avg_impact"), count("title").alias("pub_count")) \
    .orderBy(desc("avg_impact")) \
    .show(5)

print("\n✅ ETL COMPLETE.")
spark.stop()


>>> Initializing Spark Session...
    (If this takes >2 mins, it is downloading MongoDB connectors)
✅ Spark Session Active: v3.4.3

>>> 1. READING FROM MONGODB (aci.articles)...
   - Loaded 272 raw articles.

>>> 2. TRANSFORMING DATA...

>>> 3. LOADING TO DATA WAREHOUSE (aci.fact_publications)...
   - Success. Data stored in MongoDB.

>>> 4. GENERATING HDFS STRUCTURE AT: C:\Users\Dell\Desktop\Projet_BI\S2_ApacheAnalysis\hdfs_data
   - HDFS Folder Structure Created ✅

        SPARK ANALYSIS RESULTS        

📊 1. Top Countries by Impact Factor:
+-------+-----------------+---------+
|country|       avg_impact|pub_count|
+-------+-----------------+---------+
|  China|9.286122448979588|       49|
|     UK|        8.7134375|       32|
| France|8.413749999999999|       24|
|    USA|8.362153846153845|       65|
|Morocco|7.951428571428571|       14|
+-------+-----------------+---------+
only showing top 5 rows


✅ ETL COMPLETE.
